In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from chatbot_test import ChatBotTs, load_api_key
from OktSentiment import Sentiment
from tarot import TarotBot

app = Flask(__name__)
CORS(app)

# ChatBot 인스턴스 생성
api_key = load_api_key()
if not api_key:
    raise ValueError("API key not found or invalid. Please check your api_key.txt file.")
chatbot = ChatBotTs(api_key, 'C:\\Users\\SMHRD\\Documents\\카카오톡 받은 파일\\유형,대처,유형설명(마카님 삭제).csv')  # CSV 파일 경로를 적절히 수정하세요
#C:\\Users\\SMHRD\\Documents\\카카오톡 받은 파일\\유형,대처,유형설명(마카님 삭제).csv
#c:/Users/SMHRD/원인-대처종합.csv

# Sentiment 인스턴스 생성
sentiment_analyzer = Sentiment()

@app.route("/chatbot", methods=["POST"])
def chat():
    data = request.json
    print("Received data:", data)
    user_nick = data.get("user_nick")
    user_input = data.get("input")
    
    if user_input:
        try:
            response = chatbot.process_input(user_nick, user_input)
            if response.get("isProblem"):
                # 형태소 분석 및 감정 분석 수행
                sentiment_results = sentiment_analyzer.parse([user_input])
                # 결과가 딕셔너리 형태로 바꾸고 소수점 소수점 둘쨰자리까지
                if 'polarity' in sentiment_results and len(sentiment_results['polarity']) > 0:
                    sentiment_results = sentiment_results['polarity'][0]
                    sentiment_results = {key: round(value * 100, 2) for key, value in sentiment_results.items()}

                response.update({
                    "sentiment": sentiment_results
                })

            return jsonify(response)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return jsonify({"error": f"An error occurred while processing your request: {str(e)}"}), 500
    else:
        return jsonify({"error": "No input provided"}), 400

# 타로카드 라우트
@app.route("/api/interpret",methods=["POST"])

def tarot_card():
    print(request.headers)
    print(request.data)
    
    data = request.get_json()
    cards = data.get("cards")
    user_mode = data.get("user_mode")
    user_input = data.get("user_input")
    user_select = data.get("user_select")
    print("recived data: ", data)
    if not cards or len(cards) != 3:
        return jsonify({"error": "카드를 다시 선택해 주세요."}), 400
            
    # key_path = "c:\\Users\\SMHRD\\api.txt"
    key_path = "C:\\Users\\SMHRD\\Desktop\\ky_api.txt"
    try:
        tarot = TarotBot(api_key_path=key_path)
        if user_mode == "일반":
            response = tarot.general_reading(user_select,cards)
            
        elif user_mode == "심리":
            print(user_input,cards)
            response = tarot.interpret_cards(user_input,cards)
        else:
            return jsonify({"error":"Invalid user_mode value."})
        return response
    except Exception as e:
        print("Error",str(e))
        return jsonify({"error":str(e)}), 500


if __name__ == "__main__":
    app.run(port=7000)